# Goal

My goal here is to turn the messy birthdate and birthplace column into two nice columns with the proper formatting.

I will probably use regex and maybe the timestrip thing I tried out before.

In [1]:
import pandas as pd
import numpy as np
import re

Let's do an example with a random occupation csv.

In [2]:
df = pd.read_csv('hi_its_philosophers_df.csv')

In [3]:
df.head()

,name,info,occupation
0,Michael_Haneke,Gender:MaleDate of Birth:23 March 1942Birth p...,r_philosophers
1,Noam_Chomsky,Gender:MaleDate of Birth:7 December 1928 - 07...,r_philosophers
2,Albert_Camus,Gender:MaleDate of Birth:7 November 1913 - 02...,r_philosophers
3,Mahatma_Gandhi,Gender:MaleDate of Birth:2 October 1869 - 07:...,r_philosophers
4,Jean-Jacques_Rousseau,Gender:MaleDate of Birth:28 June 1712 - 10:00...,r_philosophers


Even though the name and occupation columns aren't perfect either, I can worry about those later.

For now, let's fix the info column by extracting the birthdate and birthtime, putting them in the correct format, and creating a new pair of columns for them.

In [8]:
b_info = df['info'][1]
b_info

'Gender:MaleDate of Birth:7 December  1928 - 07:00hBirth place:Philadelphia, PennsylvaniaCountry:US, United StatesAge:91 years'

In [38]:
b_info_mod = ''
b_info_mod = b_info.replace('Gender:MaleDate of Birth:', '')
b_info_mod = b_info_mod.split('Birth place:')
b_info_mod_time = b_info_mod[0]
b_info_mod_time = b_info_mod_time.split(' - ')
print(b_info_mod_time)
b_info_mod_place = b_info_mod[1].split('Age:')
b_info_mod_place = b_info_mod_place[0]
b_info_mod_place = b_info_mod_place.split('Country:')
print(b_info_mod_place)

['7 December  1928', '07:00h']
['Philadelphia, Pennsylvania', 'US, United States']


In [143]:
def get_birthtime_info(row):
    b_info = df['info'][row]
    b_info_mod = ''
    b_info_mod = b_info.replace('Gender:MaleDate of Birth:', '')
    b_info_mod = b_info_mod.split('Birth place:')
    b_info_mod_time = b_info_mod[0]
    b_info_mod_time = b_info_mod_time.split(' - ')
    #print(b_info_mod_time)
    #print('...end of get_birthtime_info')
    return b_info_mod_time

def get_birthplace_info(row):
    b_info = df['info'][row]
    b_info_mod = ''
    if 'Gender:FemaleDate of Birth:' in b_info:
        b_info_mod = b_info.replace('Gender:FemaleDate of Birth:', '')
    elif 'Gender:MaleDate of Birth:' in b_info:
        b_info_mod = b_info.replace('Gender:MaleDate of Birth:', '')
    else:
        return None
    b_info_mod = b_info_mod.split('Birth place:')
    b_info_mod_place = b_info_mod[1].split('Age:')
    b_info_mod_place = b_info_mod_place[0]
    b_info_mod_place = b_info_mod_place.split('Country:')
    #print(b_info_mod_place)
    #print('...end of get_birthplace_info')
    return b_info_mod_place

In [61]:
for i in range(5):
    get_birthtime_info(i)
    get_birthplace_info(i)

['23 March  1942']
...end of get_birthtime_info
['Munich', 'DE, Germany']
...end of get_birthplace_info
['7 December  1928', '07:00h']
...end of get_birthtime_info
['Philadelphia, Pennsylvania', 'US, United States']
...end of get_birthplace_info
['7 November  1913', '02:00h']
...end of get_birthtime_info
['Mondovi', 'DZ, Algeria']
...end of get_birthplace_info
['2 October  1869', '07:08h']
...end of get_birthtime_info
['Porbandar', 'IN, India']
...end of get_birthplace_info
['28 June  1712', '10:00h']
...end of get_birthtime_info
['Geneva', 'CH, Switzerland']
...end of get_birthplace_info


We now have a list with birthtime and birthdate, as well as a list with birthcity and birthcountry.



(((((Now let's create a function which inputs these lists and outputs a something in the proper format for using FlatLib.

This means that you can use GeoPy in there to get lon/lat.)))))

Actually, let's just output a birthtime that is properly formatted for FlatLib, and then a birthplace which is properly formatted for GeoPy -- we can do the GeoPy conversion-to-lon/-lat on the actual dataframe later.

In [102]:
def convert_month_name_to_number(month):
    month = month.lower()
    if month == 'january':
        return 1
    elif month == 'february':
        return 1
    elif month == 'march':
        return 1
    elif month == 'april':
        return 1
    elif month == 'may':
        return 1
    elif month == 'june':
        return 1
    elif month == 'july':
        return 1
    elif month == 'august':
        return 1
    elif month == 'september':
        return 1
    elif month == 'october':
        return 1
    elif month == 'november':
        return 1
    elif month == 'december':
        return 1
    else:
        print('Not_a_month')
        return None

def time_formatter(time_lst):
    if len(time_lst) == 2:
        first = time_lst[0]
        month = " ".join(re.split("[^a-zA-Z]*", first))
        month = month.replace(' ', '')
        first = first.replace(' ', '')
        temp = first.split(month)
        day = temp[0]
        year = temp[1]
        time = time_lst[1]
        time = time[:-1]
        ans = [year, convert_month_name_to_number(month), day, time]
        print(ans)
        return ans
    elif len(time_lst) == 1:
        first = time_lst[0]
        month = " ".join(re.split("[^a-zA-Z]*", first))
        month = month.replace(' ', '')
        first = first.replace(' ', '')
        temp = first.split(month)
        day = temp[0]
        year = temp[1]
        ans = [year, convert_month_name_to_number(month), day]
        print(ans)
        return ans
    return 'No_time_info'
        
    
def place_formatter(place_lst):
    city_info = place_lst[0]
    country_info = place_lst[1]
    if ',' in city_info:
        city_info = city_info.split(', ')
    else:
        city_info = [city_info]
    if ',' in country_info:
        country_info = country_info.split(', ')
        country_info = country_info[1]
    if len(city_info) == 2:
        city = city_info[0]
        state = city_info[1]
    elif len(city_info) == 1:
        city = city_info[0]
        state = 'no_state'
    country = country_info
    if state == 'no_state':
        ans = [city, country]
        ans = ', '.join(ans)
    else:
        ans = [city, state, country]
        ans = ', '.join(ans)
    print(ans)
    return ans

In [114]:
for i in range(15):
    time_formatter(get_birthtime_info(i))
    place_formatter(get_birthplace_info(i))

['1942', 1, '23']
Munich, Germany
['1928', 1, '7', '07:00']
Philadelphia, Pennsylvania, United States
['1913', 1, '7', '02:00']
Mondovi, Algeria
['1869', 1, '2', '07:08']
Porbandar, India
['1712', 1, '28', '10:00']
Geneva, Switzerland
['1713', 1, '5', '17:00']
Langres, France
['1724', 1, '22', '05:00']
Königsberg, Germany
['1744', 1, '25', '23:30']
Mohrungen, Poland
['1813', 1, '5']
Copenhagen, Denmark
['1844', 1, '15', '10:00']
Roecken, Germany
['1872', 1, '18', '17:45']
Trelleck, Wales, United Kingdom
['1928', 1, '16', '12:15']
Chicago, Illinois, United States
['1788', 1, '22', '12:00']
Gdańsk, Poland
['1818', 1, '5', '02:00']
Trier, Germany


IndexError: list index out of range

So now I'm gonna convert the time list into the right format for FlatLib.

In [ ]:
#OLD CODE FROM A WHILE BACK:
'''
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.chart import Chart
from flatlib import const


date = Datetime('1997/07/02', '16:07', '-07:00')
#date.jd
print(date.jd)

pos = GeoPos('37n7644', '121w9540')
print(pos.lat,pos.lon)

chart = Chart(date, pos)

sun = chart.getObject(const.SUN)
print(sun)

moon = chart.get(const.MOON)
print(moon)



print(sun.lon)

print(sun.lat)

print(sun.sign)

print(sun.signlon)

print(sun.lonspeed)

print(sun.orb())

print(sun.meanMotion())

print(sun.movement())

print(sun.gender())

print(sun.element())

print(sun.isFast())
'''

From the above code, I see that I need:

-xxxx/xx/xx (year, month, day) in numbers

-xx:xx (time) in 24-hour time

-'-xx:xx' or '+xx:xx' (time) for time-change

Seeing as I need the time-change... I'm gonna work on the place list first.

So I'll determine the proper location format for GeoPy.

Ok, I just check the proper location format for GeoPy... if I turn my list into a string, I'll be good.

In [104]:
def make_time_column(row):
    return time_formatter(get_birthtime_info(row))

def make_place_column(row):
    return place_formatter(get_birthplace_info(row))

In [105]:
copy_df = df

In [106]:
copy_df.head()

,name,info,occupation
0,Michael_Haneke,Gender:MaleDate of Birth:23 March 1942Birth p...,r_philosophers
1,Noam_Chomsky,Gender:MaleDate of Birth:7 December 1928 - 07...,r_philosophers
2,Albert_Camus,Gender:MaleDate of Birth:7 November 1913 - 02...,r_philosophers
3,Mahatma_Gandhi,Gender:MaleDate of Birth:2 October 1869 - 07:...,r_philosophers
4,Jean-Jacques_Rousseau,Gender:MaleDate of Birth:28 June 1712 - 10:00...,r_philosophers


In [109]:
copy_df['count'] = np.arange(len(copy_df))

In [110]:
copy_df.head()

,name,info,occupation,count
0,Michael_Haneke,Gender:MaleDate of Birth:23 March 1942Birth p...,r_philosophers,0
1,Noam_Chomsky,Gender:MaleDate of Birth:7 December 1928 - 07...,r_philosophers,1
2,Albert_Camus,Gender:MaleDate of Birth:7 November 1913 - 02...,r_philosophers,2
3,Mahatma_Gandhi,Gender:MaleDate of Birth:2 October 1869 - 07:...,r_philosophers,3
4,Jean-Jacques_Rousseau,Gender:MaleDate of Birth:28 June 1712 - 10:00...,r_philosophers,4


In [113]:
copy_df.head(20)

,name,info,occupation,count
0,Michael_Haneke,Gender:MaleDate of Birth:23 March 1942Birth p...,r_philosophers,0
1,Noam_Chomsky,Gender:MaleDate of Birth:7 December 1928 - 07...,r_philosophers,1
2,Albert_Camus,Gender:MaleDate of Birth:7 November 1913 - 02...,r_philosophers,2
3,Mahatma_Gandhi,Gender:MaleDate of Birth:2 October 1869 - 07:...,r_philosophers,3
4,Jean-Jacques_Rousseau,Gender:MaleDate of Birth:28 June 1712 - 10:00...,r_philosophers,4
5,Denis_Diderot,Gender:MaleDate of Birth:5 October 1713 - 17:...,r_philosophers,5
6,Immanuel_Kant,Gender:MaleDate of Birth:22 April 1724 - 05:0...,r_philosophers,6
7,Johann_Gottfried_Herder,Gender:MaleDate of Birth:25 August 1744 - 23:...,r_philosophers,7
8,Soeren_Kierkegaard,Gender:MaleDate of Birth:5 May 1813Birth plac...,r_philosophers,8
9,Friedrich_Nietzsche,Gender:MaleDate of Birth:15 October 1844 - 10...,r_philosophers,9


In [119]:
copy_df.iloc[14].info

'Gender:FemaleDate of Birth:12 August  1831 - 02:17hBirth place:Ekaterinoslav, DniproCountry:UA, UkraineAge:† 59 years'

In [122]:
import string

In [126]:
def adios_weird_characters(i):
    s = copy_df.iloc[i].info
    printable = set(string.printable)
    return ''.join(filter(lambda x: x in printable, s))

In [146]:
adios_weird_characters(1)

'Gender:MaleDate of Birth:7 December  1928 - 07:00hBirth place:Philadelphia, PennsylvaniaCountry:US, United StatesAge:91 years'

In [145]:
adios_weird_characters(13)

'Gender:MaleDate of Birth:5 May  1818 - 02:00hBirth place:TrierCountry:DE, GermanyAge: 64 years'

In [128]:
adios_weird_characters(14)

'Gender:FemaleDate of Birth:12 August  1831 - 02:17hBirth place:Ekaterinoslav, DniproCountry:UA, UkraineAge: 59 years'

In [129]:
copy_df['info_norm'] = copy_df['count'].apply(adios_weird_characters)

In [132]:
copy_df_mod = copy_df.drop(['info'], axis=1)
copy_df_mod.head(15)

,name,occupation,count,info_norm
0,Michael_Haneke,r_philosophers,0,Gender:MaleDate of Birth:23 March 1942Birth p...
1,Noam_Chomsky,r_philosophers,1,Gender:MaleDate of Birth:7 December 1928 - 07...
2,Albert_Camus,r_philosophers,2,Gender:MaleDate of Birth:7 November 1913 - 02...
3,Mahatma_Gandhi,r_philosophers,3,Gender:MaleDate of Birth:2 October 1869 - 07:...
4,Jean-Jacques_Rousseau,r_philosophers,4,Gender:MaleDate of Birth:28 June 1712 - 10:00...
5,Denis_Diderot,r_philosophers,5,Gender:MaleDate of Birth:5 October 1713 - 17:...
6,Immanuel_Kant,r_philosophers,6,Gender:MaleDate of Birth:22 April 1724 - 05:0...
7,Johann_Gottfried_Herder,r_philosophers,7,Gender:MaleDate of Birth:25 August 1744 - 23:...
8,Soeren_Kierkegaard,r_philosophers,8,Gender:MaleDate of Birth:5 May 1813Birth plac...
9,Friedrich_Nietzsche,r_philosophers,9,Gender:MaleDate of Birth:15 October 1844 - 10...


In [136]:
copyd_df_mod = copy_df_mod.rename(columns={"info_norm": "info"}, inplace=True)
copy_df_mod.head()

,name,occupation,count,info
0,Michael_Haneke,r_philosophers,0,Gender:MaleDate of Birth:23 March 1942Birth p...
1,Noam_Chomsky,r_philosophers,1,Gender:MaleDate of Birth:7 December 1928 - 07...
2,Albert_Camus,r_philosophers,2,Gender:MaleDate of Birth:7 November 1913 - 02...
3,Mahatma_Gandhi,r_philosophers,3,Gender:MaleDate of Birth:2 October 1869 - 07:...
4,Jean-Jacques_Rousseau,r_philosophers,4,Gender:MaleDate of Birth:28 June 1712 - 10:00...


In [141]:
copy_df_mod.iloc[14].info

'Gender:FemaleDate of Birth:12 August  1831 - 02:17hBirth place:Ekaterinoslav, DniproCountry:UA, UkraineAge: 59 years'

In [144]:
copy_df_mod['birthtime'] = copy_df_mod['count'].apply(make_time_column)
#copy_df['birthtime'] = copy_df['count'].apply(make_time_column)

['1942', 1, '23']
['1928', 1, '7', '07:00']
['1913', 1, '7', '02:00']
['1869', 1, '2', '07:08']
['1712', 1, '28', '10:00']
['1713', 1, '5', '17:00']
['1724', 1, '22', '05:00']
['1744', 1, '25', '23:30']
['1813', 1, '5']
['1844', 1, '15', '10:00']
['1872', 1, '18', '17:45']
['1928', 1, '16', '12:15']
['1788', 1, '22', '12:00']
['1818', 1, '5', '02:00']


IndexError: list index out of range

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['birthplace'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)